<a href="https://colab.research.google.com/github/Miguel1897/01-Tarea-Repaso-ReactJs/blob/master/DeteccionEnfermedades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importación de Librerías



In [3]:
# librerías necesarias
!pip install -q kaggle
!pip install torch torchvision

# Importa las librerías
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt


Configuración de la API de Kaggle y Descarga del Dataset


In [11]:
# Configura el archivo de autenticación kaggle.json
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Descarga el dataset desde Kaggle
!kaggle datasets download -d kanishk3813/pathogen-dataset --unzip

# Verifica la ubicación del dataset descargado
!ls /content

# Ajusta las rutas para cargar el dataset en PyTorch
from torchvision import datasets, transforms
import os

# Define la ruta del directorio según el nombre de la carpeta descomprimida
dataset_dir = '/content/pathogen-dataset'  # Cambia esto si el nombre de la carpeta es diferente

# Transforma las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])



Dataset URL: https://www.kaggle.com/datasets/kanishk3813/pathogen-dataset
License(s): apache-2.0
 99% 1.41G/1.43G [00:14<00:00, 109MB/s]
100% 1.43G/1.43G [00:14<00:00, 108MB/s]
kaggle.json  pathogen  sample_data


Preparación del Dataset

In [15]:
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import os

# Especifica la ruta del dataset descargado
dataset_dir = '/content/pathogen'  # Ruta donde están las carpetas de clases

# Define las transformaciones para las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionar las imágenes a 128x128
    transforms.ToTensor()  # Convertir las imágenes a tensores
])

# Cargar el dataset completo con todas las clases
dataset = datasets.ImageFolder(dataset_dir, transform=transform)

# Verificar las clases detectadas en el dataset
print("Clases:", dataset.classes)

# Dividir el dataset en entrenamiento y validación (80% entrenamiento, 20% validación)
train_size = int(0.8 * len(dataset))  # 80% para entrenamiento
valid_size = len(dataset) - train_size  # El resto para validación

# Dividir el dataset en dos
train_data, valid_data = random_split(dataset, [train_size, valid_size])

# Crear los DataLoaders para entrenamiento y validación
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32)

# Verificar la cantidad de muestras en los DataLoaders
print(f'Tamaño del conjunto de entrenamiento: {len(train_data)}')
print(f'Tamaño del conjunto de validación: {len(valid_data)}')

# Verificar algunas clases
print(f'Primeras clases de entrenamiento: {train_data.indices[:5]}')


Clases: ['Bacteria', 'Fungi', 'Healthy', 'Pests', 'Virus']
Tamaño del conjunto de entrenamiento: 31997
Tamaño del conjunto de validación: 8000
Primeras clases de entrenamiento: [33078, 31157, 17748, 22576, 1109]


Definición del Modelo CNN

In [16]:
import torch
import torch.nn as nn

class PlagaDetectorCNN(nn.Module):
    def __init__(self):
        super(PlagaDetectorCNN, self).__init__()

        # Capas convolucionales
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)  # Conv1: 3 canales de entrada, 32 canales de salida
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)  # Conv2: 32 canales de entrada, 64 de salida
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # Conv3: 64 canales de entrada, 128 de salida

        # Capa de max-pooling
        self.pool = nn.MaxPool2d(2, 2)  # Tamaño del kernel 2x2

        # Capas completamente conectadas
        self.fc1 = nn.Linear(128 * 16 * 16, 512)  # Capa densa (128x16x16 es la dimensión de salida de las convoluciones)
        self.fc2 = nn.Linear(512, len(train_data.classes))  # La salida final tiene tantas clases como train_data.classes

    def forward(self, x):
        # Aplicamos las convoluciones seguidas de ReLU y MaxPooling
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))

        # Aplanamos la salida para pasarla a la capa totalmente conectada
        x = x.view(-1, 128 * 16 * 16)  # Aplanar: las dimensiones deben coincidir con la capa fc1

        # Capas completamente conectadas
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)

        return x


Configuración del Dispositivo (CPU o GPU)

In [19]:
# Suponiendo que train_data_full es el conjunto de datos completo ImageFolder
train_data_full = datasets.ImageFolder(os.path.join(dataset_dir), transform=transform)

# Obtener el número de clases
num_classes = len(train_data_full.classes)

# Definir el modelo usando el número de clases
class PlagaDetectorCNN(nn.Module):
    def __init__(self, num_classes):
        super(PlagaDetectorCNN, self).__init__()

        # Capas convolucionales
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)
        self.fc2 = nn.Linear(512, num_classes)  # Usar num_classes aquí

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Ahora crear el modelo con el número de clases correcto
model = PlagaDetectorCNN(num_classes).to(device)


Definición de Funciones de Entrenamiento y Evaluación

In [20]:
import torch
import torch.optim as optim
import torch.nn as nn

# Define el criterio de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Función para entrenar el modelo
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)  # Mover a GPU/CPU
            optimizer.zero_grad()  # Limpiar gradientes previos
            outputs = model(images)  # Obtener las predicciones del modelo
            loss = criterion(outputs, labels)  # Calcular la pérdida
            loss.backward()  # Calcular el gradiente
            optimizer.step()  # Actualizar los parámetros
            running_loss += loss.item()  # Acumular la pérdida de la época
        # Promediar la pérdida de la época
        print(f"Epoch {epoch+1}, Pérdida promedio: {running_loss/len(train_loader)}")

# Función para evaluar el modelo en el conjunto de validación
def evaluate_model(model, valid_loader):
    model.eval()  # Modo evaluación (desactiva dropout, etc.)
    total, correct = 0, 0
    running_loss = 0.0
    with torch.no_grad():  # No calcular gradientes durante la validación
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)  # Obtener las predicciones del modelo
            loss = criterion(outputs, labels)  # Calcular la pérdida de validación
            running_loss += loss.item()  # Acumular la pérdida de validación
            _, predicted = torch.max(outputs, 1)  # Obtener las predicciones (la clase con mayor probabilidad)
            total += labels.size(0)  # Total de muestras
            correct += (predicted == labels).sum().item()  # Comparar con las etiquetas reales

    # Imprimir métricas
    print(f"Pérdida en validación: {running_loss/len(valid_loader)}")
    print(f"Precisión en validación: {100 * correct / total}%")


Entrenamiento del Modelo


In [ ]:
train_model(model, train_loader, criterion, optimizer, epochs=10)
evaluate_model(model, valid_loader)



Uso del Modelo para Predicciones


In [ ]:
from PIL import Image
import torch
from torchvision import transforms
import matplotlib.pyplot as plt

def predict_image(image_path, model):
    model.eval()  # Modo evaluación
    # Transforma la imagen para que sea compatible con el modelo
    transform = transforms.Compose([transforms.Resize((128, 128)), transforms.ToTensor()])
    image = Image.open(image_path)  # Abre la imagen
    image = transform(image).unsqueeze(0).to(device)  # Aplica las transformaciones y añade la dimensión de batch
    output = model(image)  # Realiza la predicción
    _, predicted = torch.max(output, 1)  # Obtén la clase con la mayor probabilidad
    return train_data.classes[predicted.item()]  # Devuelve el nombre de la clase

# Ruta de la imagen de prueba
image_path = '/content/pathogen/test/image.jpg'  # Cambia este path por el correcto

# Realiza la predicción
prediccion = predict_image(image_path, model)
print(f"Plaga detectada: {prediccion}")

# Muestra la imagen junto con la predicción
plt.imshow(Image.open(image_path))
plt.title(f"Predicción: {prediccion}")
plt.show()

